# COVID-19 Data Exploration Portfolio

<div>
<img src=https://thanhtra.com.vn/data/images/0/2021/12/05/congdinh/blue-covid-banner.jpg width="1500">
</div>

# Introduction

In this project, we delve into the extensive dataset of COVID-19, utilizing a range of SQL skills to extract meaningful insights. The dataset encompasses vital statistics on COVID-19 cases, deaths, population demographics, and vaccination progress across various locations and continents.

## Skills Utilized

- **Joins:** Connecting data from multiple sources for comprehensive analysis.

- **Common Table Expressions (CTE's):** Simplifying complex queries and calculations.

- **Temporary Tables:** Storing interim results for efficient data manipulation.

- **Window Functions:** Performing computations over specified subsets of data.

- **Aggregate Functions:** Summarizing data to reveal trends and patterns.

- **Creating Views:** Organizing data for future reference and visualization.

- **Converting Data Types:** Ensuring data compatibility and accuracy.

## Project Highlights
1. **Initial Data Exploration:**

- A look at the core dataset focusing on COVID-19 cases and population statistics.

- Understanding the progression of total cases, deaths, and infection rates.

2. **Country-specific Analysis:**

- Examining the likelihood of death if infected and the percentage of population infected.

- Identifying countries with the highest infection rates and death counts.

3. **Continental Breakdown:**

- Analyzing continents with the highest death counts per population.

- Understanding the global impact on different regions.

4. **Vaccination Progress:**

- Tracking the percentage of the population vaccinated.

- Utilizing various methods like CTEs and temporary tables for calculations.

5. **Data Storage and Future Use:**

- Creating a view for easy access to vaccination and demographic data.

- Setting the stage for insightful visualizations and further analysis.


This project not only showcases SQL proficiency but also aims to uncover trends, disparities, and progress in the fight against COVID-19. Join us on this data-driven journey as we unravel the story behind the numbers.

# Overview of the dataset

In this project, we focus on analyzing specific aspects of the extensive [COVID-19 dataset](https://ourworldindata.org/covid-deaths) provided by Our World in Data. This dataset is diligently maintained by Our World in Data, with daily updates to ensure it reflects the latest information relevant to the COVID-19 pandemic. It encompasses a diverse range of crucial metrics sourced from reputable institutions, guaranteeing accuracy and reliability.

Our analysis hones in on the following key variables:

- **Confirmed Cases:** Total and new confirmed cases, smoothed averages, and cases per million people.
- **Confirmed Deaths:** Total and new deaths attributed to COVID-19, smoothed averages, and deaths per million people.
- **Vaccinations:** Total doses administered, people vaccinated, fully vaccinated, and booster doses.

The dataset we're using contains 390,786 rows spanning from January 1, 2020, to April 18, 2024. It is divided into two CSV files for easier analysis:

- CovidDeaths.csv: Contains the data on confirmed cases and deaths.
- CovidVaccinations.csv: Contains the data on vaccinations.

For further insights, you can explore the complete dataset [here](https://github.com/owid/covid-19-data/tree/master/public/data).

# Connect to the database

We will use the SQLite database in this project to easily work with SQL on Google Colab.

In [46]:
import sqlite3

# Connect to an SQLite database
conn = sqlite3.connect('covid_data.db')

In [47]:
%%capture
# Install ipython-sql
!pip install ipython-sql

In [48]:
# Load the SQL extension
%load_ext sql

# Create a SQLite database
%sql sqlite:///covid_data.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# Load the dataset

We will import the dataset from CSV files into the SQLite database we've created.

In [49]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Datasets/CovidDeaths.csv")
# df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y').dt.date

df.to_sql("COVID_DEATHS", conn, if_exists="append", chunksize=100, index=False, method="multi")

390786

In [51]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/CovidVaccinations.csv")
# df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y').dt.date

df.to_sql("COVID_VACCINATIONS", conn, if_exists="append", chunksize=100, index=False, method="multi")

390786

# Data exploration

## COVID cases and deaths

Let's start with the `COVID_DEATHS` dataset, which contains the data on confirmed cases and deaths.

In [52]:
%%sql
SELECT continent, location, date, population, total_cases, new_cases, total_deaths, new_deaths
FROM COVID_DEATHS
ORDER BY location, date
LIMIT 10

 * sqlite:///covid_data.db
Done.


continent,location,date,population,total_cases,new_cases,total_deaths,new_deaths
Asia,Afghanistan,1/1/2021,41128772,51848.0,0.0,2158.0,0.0
Asia,Afghanistan,1/1/2021,41128772,51848.0,0.0,2158.0,0.0
Asia,Afghanistan,1/1/2022,41128772,157902.0,0.0,7352.0,0.0
Asia,Afghanistan,1/1/2022,41128772,157902.0,0.0,7352.0,0.0
Asia,Afghanistan,1/1/2023,41128772,207579.0,257.0,7849.0,4.0
Asia,Afghanistan,1/1/2023,41128772,207579.0,257.0,7849.0,4.0
Asia,Afghanistan,1/1/2024,41128772,230375.0,0.0,7973.0,0.0
Asia,Afghanistan,1/1/2024,41128772,230375.0,0.0,7973.0,0.0
Asia,Afghanistan,1/10/2020,41128772,None,0.0,None,0.0
Asia,Afghanistan,1/10/2020,41128772,None,0.0,None,0.0


We can observe that the dataset is updated weekly, with new cases and new deaths reported only on Sundays. Therefore, in this analysis, the dates of events (if mentioned) are only accurate in weeks.

Additionally, there seems to be an issue with the date column as the ordering does not function correctly in the previous query. Let's examine the table schema to identify the problem.

In [53]:
%%sql
PRAGMA table_info(COVID_DEATHS)

 * sqlite:///covid_data.db
Done.


cid,name,type,notnull,dflt_value,pk
0,iso_code,TEXT,0,None,0
1,continent,TEXT,0,None,0
2,location,TEXT,0,None,0
3,date,TEXT,0,None,0
4,population,INTEGER,0,None,0
5,total_cases,REAL,0,None,0
6,new_cases,REAL,0,None,0
7,new_cases_smoothed,REAL,0,None,0
8,total_deaths,REAL,0,None,0
9,new_deaths,REAL,0,None,0


We see that the `date` column is now stored as a string, which is not the correct datatype for a date. To fix this, we will convert this column to a `DATE` datatype.

Next, we'll save the processed `date` column along with the other columns we're interested in into a temporary table called `COVID_DEATHS_PROCESSED` for future analysis.

In [54]:
%%sql
DROP TABLE IF EXISTS COVID_DEATHS_PROCESSED;

CREATE TABLE COVID_DEATHS_PROCESSED
(
    continent TEXT,
    location TEXT,
    date DATE,
    population REAL,
    total_cases REAL,
    new_cases REAL,
    total_deaths REAL,
    new_deaths REAL
);

INSERT INTO COVID_DEATHS_PROCESSED
WITH date_split AS (
    SELECT
      continent,
      location,
      SUBSTR(date, 1, INSTR(date, '/') - 1) AS month,
      SUBSTR(SUBSTR(date, INSTR(date, '/') + 1), 1, INSTR(SUBSTR(date, INSTR(date, '/') + 1), '/') - 1) AS date,
      SUBSTR(SUBSTR(date, INSTR(date, '/') + 1), INSTR(SUBSTR(date, INSTR(date, '/') + 1), '/') + 1) AS year,
      population,
      total_cases,
      new_cases,
      total_deaths,
      new_deaths
    FROM COVID_DEATHS
),
date_normalize AS (
    SELECT
      continent,
      location,
      year,
      CASE WHEN LENGTH(month) == 1 THEN '0' || month ELSE month END AS month,
      CASE WHEN LENGTH(date) == 1 THEN '0' || date ELSE date END AS date,
      population,
      total_cases,
      new_cases,
      total_deaths,
      new_deaths
    FROM date_split
)

SELECT
    continent,
    location,
    DATE(year || '-' || month || '-' || date) AS date,
    population,
    total_cases,
    new_cases,
    total_deaths,
    new_deaths
FROM date_normalize
ORDER BY location, date

 * sqlite:///covid_data.db
Done.
Done.
781572 rows affected.


[]

Let's take a look at our new table.

In [55]:
%%sql
SELECT *
FROM COVID_DEATHS_PROCESSED
LIMIT 10

 * sqlite:///covid_data.db
Done.


continent,location,date,population,total_cases,new_cases,total_deaths,new_deaths
Asia,Afghanistan,2020-01-05,41128772.0,None,0.0,None,0.0
Asia,Afghanistan,2020-01-05,41128772.0,None,0.0,None,0.0
Asia,Afghanistan,2020-01-06,41128772.0,None,0.0,None,0.0
Asia,Afghanistan,2020-01-06,41128772.0,None,0.0,None,0.0
Asia,Afghanistan,2020-01-07,41128772.0,None,0.0,None,0.0
Asia,Afghanistan,2020-01-07,41128772.0,None,0.0,None,0.0
Asia,Afghanistan,2020-01-08,41128772.0,None,0.0,None,0.0
Asia,Afghanistan,2020-01-08,41128772.0,None,0.0,None,0.0
Asia,Afghanistan,2020-01-09,41128772.0,None,0.0,None,0.0
Asia,Afghanistan,2020-01-09,41128772.0,None,0.0,None,0.0


In [56]:
%%sql
PRAGMA table_info(COVID_DEATHS_PROCESSED)

 * sqlite:///covid_data.db
Done.


cid,name,type,notnull,dflt_value,pk
0,continent,TEXT,0,None,0
1,location,TEXT,0,None,0
2,date,DATE,0,None,0
3,population,REAL,0,None,0
4,total_cases,REAL,0,None,0
5,new_cases,REAL,0,None,0
6,total_deaths,REAL,0,None,0
7,new_deaths,REAL,0,None,0


Everything looks good. Now, let's explore the number of unique locations and continents in our dataset using the processed table.

In [57]:
%%sql
SELECT DISTINCT continent, location
FROM COVID_DEATHS_PROCESSED
ORDER BY 1, 2
LIMIT 10

 * sqlite:///covid_data.db
Done.


continent,location
None,Africa
None,Asia
None,Europe
None,European Union
None,High income
None,Low income
None,Lower middle income
None,North America
None,Oceania
None,South America


We can observe that there are rows where the `continent` is null, and the `location` is the continent itself (such as Asia, Africa, Europe, etc.), or descriptors like High income, Low income, Lower middle income, Upper middle income, or World.

For our further analysis in this project, to obtain accurate numbers for each country, we will filter out the rows where the `continent` is null. If we need to gather the numbers for each continent, we will use the rows where the `continent` is null and the `location` is the respective continent.

### Insights for the whole world

Let's examine the global total number of COVID-19 cases and deaths.

In [58]:
# Let's look at the global numbers
%%sql
SELECT
  MAX(total_cases) AS global_total_cases,
  MAX(total_deaths) AS global_total_deaths,
  MAX(total_deaths) / MAX(total_cases) * 1000 AS global_death_percentage
FROM COVID_DEATHS_PROCESSED
WHERE location == 'World'

 * sqlite:///covid_data.db
Done.


global_total_cases,global_total_deaths,global_death_percentage
775251765.0,7043660.0,9.08564200431069


The dataset shows that approximately 775 million confirmed COVID-19 cases and 7.04 million deaths globally, resulting in a death percentage of around 9.09%.

Let's examine the trend of new COVID-19 cases and deaths worldwide over time.

In [59]:
%%sql
SELECT
  date,
  SUM(new_cases) AS global_new_cases,
  SUM(new_deaths) AS global_new_deaths
FROM COVID_DEATHS_PROCESSED
WHERE continent IS NOT NULL
  AND NOT (new_cases == 0 AND new_deaths == 0)
GROUP BY date
ORDER BY date
LIMIT 10

 * sqlite:///covid_data.db
Done.


date,global_new_cases,global_new_deaths
2020-01-05,4.0,6.0
2020-01-12,90.0,2.0
2020-01-19,180.0,4.0
2020-01-26,3792.0,112.0
2020-02-02,25076.0,620.0
2020-02-09,45970.0,1090.0
2020-02-16,62898.0,1728.0
2020-02-23,18484.0,1380.0
2020-03-01,16534.0,1030.0
2020-03-08,40414.0,1298.0



**Insights from COVID-19 Data Analysis**
1. **Rapid Increase in Cases:**
- A sharp rise in global new COVID-19 cases is observed from March 2020.
- Notable peak: February 2, 2020, with 12,538 new cases reported in a single day.

2. **Corresponding Rise in Deaths:**
- As new cases surged, new COVID-19 deaths also increased.
- Peak in new deaths: February 2, 2020, with 310 fatalities reported on the same day.

3. **Waves of Increase:**
- Waves of high new cases and deaths were evident in March 2020, July 2020, and January 2021.
- These periods signify significant challenges in controlling the spread and impact of the virus.

4. **Recent Trends:**
- Towards the end of 2021 and into 2022, a gradual decrease in new cases and deaths is observed.
- However, ongoing fluctuations suggest continued challenges despite improvements in some regions.

These insights provide a snapshot of the dynamic nature of the COVID-19 pandemic, showcasing periods of rapid escalation, peaks, and recent improvements. It underscores the importance of ongoing vigilance and strategies to combat the virus's spread and impact on a global scale.

### Insights for each continent

Let's take a closer look at the numbers of COVID-19 cases and deaths in each continent.

In [60]:
%%sql
SELECT
  location,
  MAX(population) AS population,
  MAX(total_cases) AS total_cases,
  MAX(total_deaths) AS total_deaths,
  MAX(total_cases) / MAX(population) AS covid_percentage,
  MAX(total_deaths) / MAX(total_cases) AS death_percentage
FROM COVID_DEATHS_PROCESSED
WHERE continent IS NULL
  AND location NOT IN ('World', 'High income', 'Upper middle income', 'Lower middle income', 'Low income')
GROUP BY location
ORDER BY location

 * sqlite:///covid_data.db
Done.


location,population,total_cases,total_deaths,covid_percentage,death_percentage
Africa,1426736614.0,13140491.0,259095.0,0.009210172971701699,0.01971729975691167
Asia,4721383370.0,301392304.0,1636933.0,0.06383559232132424,0.005431236890507994
Europe,744807803.0,252450811.0,2099805.0,0.3389475915573887,0.008317679755839644
European Union,450146793.0,185619587.0,1260979.0,0.4123534586638719,0.006793350962471433
North America,600323657.0,124541232.0,1661466.0,0.2074568119177086,0.01334069025429265
Oceania,45038860.0,14894138.0,32334.0,0.3306952707062301,0.0021709212040334257
South America,436816679.0,68826532.0,1354009.0,0.1575638827655663,0.01967277677160895


**Insights from COVID-19 Data Analysis by Continent**

- **Asia** has the highest total cases among the continents, with 301,392,304 reported cases. Despite this, the death percentage is relatively low at 0.54%.

- **Europe** follows closely with 252,450,811 total cases, showing a high case percentage of around 33.89%. The death percentage, while lower at 0.83%, is still notable.

- **North America** has the third highest total cases, with 124,541,232 reported. The death percentage is relatively higher at 1.33%, indicating a significant impact of COVID-19 on this continent.

- **South America** has a lower total cases compared to other continents, with 68,826,532 reported. However, the death percentage is relatively higher at 1.97%, indicating a notable impact of the virus on the population.

- Within Europe, the **European Union** has 185,619,587 total cases. The case percentage is notably high at 41.24%, with a death percentage of 0.68%.

- **Africa** has the lowest total cases among the continents, with 13,140,491 reported. However, the death percentage is relatively higher at 1.97%, indicating a significant impact on those who contract the virus.

These insights highlight Asia as the continent with the highest total cases, followed closely by Europe. Despite Europe's high case numbers, Asia has a lower death percentage. North America and South America also show significant numbers, with varying death percentages. The European Union stands out within Europe with its high case percentage. Africa has the lowest total cases but a relatively higher death percentage among the continents analyzed.


How about the total cases and deaths categorized by the country's income level?

In [61]:
%%sql
SELECT
  location,
  MAX(population) AS population,
  MAX(total_cases) AS total_cases,
  MAX(total_deaths) AS total_deaths,
  MAX(total_cases) / MAX(population) AS covid_percentage,
  MAX(total_deaths) / MAX(total_cases) AS death_percentage
FROM COVID_DEATHS_PROCESSED
WHERE continent IS NULL
  AND location IN ('High income', 'Upper middle income', 'Lower middle income', 'Low income')
GROUP BY location
ORDER BY location

 * sqlite:///covid_data.db
Done.


location,population,total_cases,total_deaths,covid_percentage,death_percentage
High income,1250514600.0,428541299.0,2983098.0,0.34269195977400024,0.006961051378154337
Low income,737604900.0,2328082.0,48045.0,0.0031562724163030913,0.020637159687674232
Lower middle income,3432097300.0,97535147.0,1341389.0,0.02841852618805417,0.01375287823167991
Upper middle income,2525921300.0,245634014.0,2667162.0,0.09724531560029206,0.010858276329759444


**Insights from COVID-19 Data Analysis by Country's Income Level**

- **High income countries** show a high total number of cases, with 428,541,299 reported. The case percentage is notably high at 34.27%, while the death percentage is relatively low at 0.70%.

- **Low income countries** have a lower total number of cases, with 2,328,082 reported. However, the death percentage is relatively higher at 2.06%, indicating a significant impact on these countries despite lower case numbers.

- **Lower middle income countries** show a moderate total number of cases at 97,535,147. The case percentage is 2.84%, with a death percentage of 1.38%.

- **Upper middle income countries** have a high total number of cases, with 245,634,014 reported. The case percentage is notably high at 9.72%, while the death percentage is 1.09%.

These insights provide a glimpse into how the COVID-19 pandemic has affected countries based on their income levels. High income countries have a notably high case percentage, while low income countries show a relatively higher death percentage despite lower case numbers. Lower and upper middle income countries fall in between, with varying case and death percentages. Understanding these trends can aid in tailoring effective public health measures for different income level countries.

### Insights for each country

Let's explore the impact of COVID-19 on each country and examine the country with the highest infection rate relative to its population.

In [62]:
%%sql
SELECT
  location,
  population,
  MAX(total_cases) AS total_cases,
  MAX(total_cases / population) AS covid_percentage
FROM COVID_DEATHS_PROCESSED
WHERE continent IS NOT NULL
GROUP BY location, population
ORDER BY covid_percentage DESC
LIMIT 10

 * sqlite:///covid_data.db
Done.


location,population,total_cases,covid_percentage
Cyprus,896007.0,690547.0,0.7706937557407476
Brunei,449002.0,343770.0,0.7656313334907194
San Marino,33690.0,25292.0,0.7507272187592757
Austria,8939617.0,6081287.0,0.6802625884308019
South Korea,51815808.0,34571873.0,0.6672070615978815
Faeroe Islands,53117.0,34658.0,0.6524841387879586
Slovenia,2119843.0,1355844.0,0.6395964229426424
Gibraltar,32677.0,20550.0,0.628882700370291
Martinique,367512.0,230354.0,0.6267931387274429
Luxembourg,647601.0,391270.0,0.6041837489441801


- **Cyprus** has the highest COVID-19 rate among the countries in the dataset, with 77.07% of its population infected.
- **Brunei** follows closely behind with a COVID-19 rate of 76.56%.
- **San Marino**, although having a much smaller population, has a COVID-19 rate of 75.07%.

These countries have experienced significant outbreaks relative to their populations. Further investigation into the factors contributing to these high rates, such as testing availability, healthcare infrastructure, and public health measures, could provide valuable insights.

Let's examine the country with the highest COVID-19 death toll per capita.


In [63]:
%%sql
SELECT
  location,
  population,
  MAX(total_deaths) AS total_deaths,
  MAX(total_deaths / population) AS death_percentage
FROM COVID_DEATHS_PROCESSED
WHERE continent IS NOT NULL
GROUP BY location, population
ORDER BY death_percentage DESC
LIMIT 10

 * sqlite:///covid_data.db
Done.


location,population,total_deaths,death_percentage
Peru,34049588.0,220831.0,0.00648557039809116
Bulgaria,6781955.0,38700.0,0.005706319195571188
Bosnia and Herzegovina,3233530.0,16388.0,0.005068145339613364
Hungary,9967304.0,49049.0,0.004920989667817897
North Macedonia,2093606.0,9977.0,0.004765462078347119
Slovenia,2119843.0,10049.0,0.004740445400909407
Croatia,4030361.0,18751.0,0.0046524368412655835
Georgia,3744385.0,17132.0,0.004575384208621709
Montenegro,627082.0,2654.0,0.004232301357717173
Czechia,10493990.0,43503.0,0.004145515671350936


- **Peru:** With a population of 34,049,588, Peru has a death rate of 0.65% and a total of 220,831 deaths.
- **Bulgaria:** With a population of 6,781,955, Bulgaria has a death rate of 0.57% and a total of 38,700 deaths.
- **Bosnia and Herzegovina:** With a population of 3,233,530, Bosnia and Herzegovina has a death rate of 0.51% and a total of 16,388 deaths.

These countries stand out for both their high death rates relative to their populations and their significant total death counts, illustrating the substantial impact of the COVID-19 pandemic on their communities.

### Insights for Vietnam

After analyzing the global, continental, and individual country numbers, let's shift our focus to the cases and deaths specifically in Vietnam.

Let's examine the relationship between `total_cases` and `population`, illustrating the percentage of our country's population affected by COVID-19 over time.

In [64]:
%%sql
SELECT
  location,
  date,
  population,
  total_cases,
  total_cases / population AS covid_percentage
FROM COVID_DEATHS_PROCESSED
WHERE continent IS NOT NULL
  AND location == 'Vietnam'
  AND NOT (new_cases == 0 AND new_deaths == 0)
ORDER BY location, date
LIMIT 10

 * sqlite:///covid_data.db
Done.


location,date,population,total_cases,covid_percentage
Vietnam,2020-01-26,98186856.0,2.0,2.0369325197661894e-08
Vietnam,2020-01-26,98186856.0,2.0,2.0369325197661894e-08
Vietnam,2020-02-02,98186856.0,6.0,6.110797559298569e-08
Vietnam,2020-02-02,98186856.0,6.0,6.110797559298569e-08
Vietnam,2020-02-09,98186856.0,13.0,1.324006137848023e-07
Vietnam,2020-02-09,98186856.0,13.0,1.324006137848023e-07
Vietnam,2020-02-16,98186856.0,16.0,1.6295460158129515e-07
Vietnam,2020-02-16,98186856.0,16.0,1.6295460158129515e-07
Vietnam,2020-03-08,98186856.0,20.0,2.0369325197661894e-07
Vietnam,2020-03-08,98186856.0,20.0,2.0369325197661894e-07


**Insights on COVID-19 Data for Vietnam**

1. **COVID-19 Cases Over Time:**
- The dataset starts with 2 cases on January 26, 2020, and shows a steady increase over time.
- By October 25, 2020, the total cases reached 1,160.
- Notable growth occurred in 2021, with 1,574,507 cases by August 1.
- A significant spike in cases was observed towards the end of 2022 and into 2023, reaching 11,524,547 by October 22.

2. **COVID-19 Percentage of Population:**
- Initially low, the COVID-19 percentage of the population increased gradually.
- By August 1, 2021, it was 0.0016%, rising to approximately 0.1184% by October 22, 2023.
- This indicates a substantial spread within the population over time.

3. **Trend in COVID-19 Cases:**
- A clear exponential growth pattern is evident, particularly from 2021 onwards.
- Rapid increases in cases were observed in the latter half of 2022 and throughout 2023.
- This suggests challenges in containing the virus and managing its spread.

The data underscores the severity and progression of COVID-19 in Vietnam, emphasizing the importance of public health measures and vaccination campaigns to combat its spread effectively. Continued vigilance and proactive measures are crucial in managing the ongoing pandemic.

Let's analyze the correlation between `total_cases` and `total_deaths`. This will provide us with insights into the mortality rate associated with COVID-19 in our country over time.

In [65]:
%%sql
SELECT
  location,
  date,
  total_cases,
  total_deaths,
  total_deaths / total_cases AS death_percentage
FROM COVID_DEATHS
WHERE continent IS NOT NULL
  AND location == 'Vietnam'
  AND NOT (new_cases == 0 AND new_deaths == 0)
ORDER BY location, date
LIMIT 10

 * sqlite:///covid_data.db
Done.


location,date,total_cases,total_deaths,death_percentage
Vietnam,1/1/2023,11525231.0,43186.0,0.0037470832471817703
Vietnam,1/1/2023,11525231.0,43186.0,0.0037470832471817703
Vietnam,1/10/2021,1514.0,35.0,0.023117569352708058
Vietnam,1/10/2021,1514.0,35.0,0.023117569352708058
Vietnam,1/15/2023,11526118.0,43186.0,0.0037467948879232366
Vietnam,1/15/2023,11526118.0,43186.0,0.0037467948879232366
Vietnam,1/16/2022,2023546.0,35609.0,0.017597326673077854
Vietnam,1/16/2022,2023546.0,35609.0,0.017597326673077854
Vietnam,1/17/2021,1537.0,35.0,0.02277163305139883
Vietnam,1/17/2021,1537.0,35.0,0.02277163305139883


**Insights on COVID-19 Data for Vietnam**

1. **Total Cases and Deaths:**
- The dataset spans from January 26, 2020, to September 6, 2023, showing a progression of the pandemic.
- The total cases have risen from 2 cases on January 26, 2020, to 116,231,82 by September 6, 2023.
- Total deaths increased from 35 on January 26, 2020, to 43,206 by September 6, 2023.

2. **Death Percentage:**
- The death percentage started at 1.75% on January 16, 2022, and decreased to 0.37% by September 6, 2023.
- It had significant fluctuations over time, with notable peaks and troughs.
- On April 17, 2022, there was a spike in the death percentage to 4.12%, indicating a period of higher mortality.
- Generally, the death percentage has been on a decreasing trend since then, suggesting improvements in healthcare or vaccination efforts.

3. **Trend Analysis:**
- The death percentage reached its highest point in April 2022 and has been on a declining trend since then.
- Total cases continued to rise steadily, especially in 2022, reaching over 100 million by September 6, 2023.
- Total deaths have also increased but at a slower rate than total cases, possibly indicating improvements in treatment or healthcare capacity.

The data analysis provides insights into the progression of COVID-19 in Vietnam, highlighting trends in total cases, total deaths, and the death percentage. While total cases continue to rise, the death percentage has shown a decreasing trend since a peak in April 2022. This suggests potential improvements in healthcare, treatment, or vaccination efforts. However, challenges remain, and the data underscores the ongoing impact of the pandemic on Vietnam.

## COVID vaccinations

Let's redirect our focus to the second table, `COVID_VACCINATIONS`.

In [66]:
%%sql
SELECT continent, location, date, total_vaccinations, new_vaccinations
FROM COVID_VACCINATIONS
ORDER BY location, date
LIMIT 10

 * sqlite:///covid_data.db
Done.


continent,location,date,total_vaccinations,new_vaccinations
Asia,Afghanistan,1/1/2021,None,None
Asia,Afghanistan,1/1/2021,None,None
Asia,Afghanistan,1/1/2022,None,None
Asia,Afghanistan,1/1/2022,None,None
Asia,Afghanistan,1/1/2023,None,None
Asia,Afghanistan,1/1/2023,None,None
Asia,Afghanistan,1/1/2024,None,None
Asia,Afghanistan,1/1/2024,None,None
Asia,Afghanistan,1/10/2020,None,None
Asia,Afghanistan,1/10/2020,None,None


We've identified an issue with the data column, much like the `COVID_DEATHS` dataset. To address this, we'll preprocess it as we did earlier with the `COVID_DEATHS` dataset and then save it in a temporary table named `COVID_VACCINATIONS_PROCESSED`.

In [67]:
%%sql
DROP TABLE IF EXISTS COVID_VACCINATIONS_PROCESSED;

CREATE TABLE COVID_VACCINATIONS_PROCESSED
(
    continent TEXT,
    location TEXT,
    date DATE,
    total_vaccinations REAL,
    new_vaccinations REAL
);

INSERT INTO COVID_VACCINATIONS_PROCESSED
WITH date_split AS (
    SELECT
      continent,
      location,
      SUBSTR(date, 1, INSTR(date, '/') - 1) AS month,
      SUBSTR(SUBSTR(date, INSTR(date, '/') + 1), 1, INSTR(SUBSTR(date, INSTR(date, '/') + 1), '/') - 1) AS date,
      SUBSTR(SUBSTR(date, INSTR(date, '/') + 1), INSTR(SUBSTR(date, INSTR(date, '/') + 1), '/') + 1) AS year,
      total_vaccinations,
      new_vaccinations
    FROM COVID_VACCINATIONS
),
date_normalize AS (
    SELECT
      continent,
      location,
      year,
      CASE WHEN LENGTH(month) == 1 THEN '0' || month ELSE month END AS month,
      CASE WHEN LENGTH(date) == 1 THEN '0' || date ELSE date END AS date,
      total_vaccinations,
      new_vaccinations
    FROM date_split
)

SELECT
    continent,
    location,
    DATE(year || '-' || month || '-' || date) AS date,
    total_vaccinations,
    new_vaccinations
FROM date_normalize
ORDER BY location, date

 * sqlite:///covid_data.db
Done.
Done.
781572 rows affected.


[]

Let's take a look at our new table.

In [68]:
%%sql
SELECT *
FROM COVID_VACCINATIONS_PROCESSED
LIMIT 10

 * sqlite:///covid_data.db
Done.


continent,location,date,total_vaccinations,new_vaccinations
Asia,Afghanistan,2020-01-05,None,None
Asia,Afghanistan,2020-01-05,None,None
Asia,Afghanistan,2020-01-06,None,None
Asia,Afghanistan,2020-01-06,None,None
Asia,Afghanistan,2020-01-07,None,None
Asia,Afghanistan,2020-01-07,None,None
Asia,Afghanistan,2020-01-08,None,None
Asia,Afghanistan,2020-01-08,None,None
Asia,Afghanistan,2020-01-09,None,None
Asia,Afghanistan,2020-01-09,None,None


We've identified another issue with our table: the `total_vaccinations` column has data in only a few rows, while most entries are empty. This could lead to a disjointed line chart if we aim to display total vaccinations on our dashboard. Additionally, the `new_vaccinations` column has many null values, which is unexpected given the increase in `total_vaccinations`.

To address these issues, we'll take the following steps:
- For every null value in the `total_vaccinations` column, we will replace it with the nearest non-null value of `total_vaccinations` for that country.
- We will then calculate the `new_vaccinations` by subtracting two consecutive `total_vaccinations` values.

Finally, we will save the processed data in a new temporary table called `COVID_VACCINATIONS_PROCESSED_V2`.

In [69]:
%%sql
DROP TABLE IF EXISTS COVID_VACCINATIONS_PROCESSED_V2;

CREATE TABLE COVID_VACCINATIONS_PROCESSED_V2
(
    continent TEXT,
    location TEXT,
    date DATE,
    total_vaccinations REAL,
    new_vaccinations REAL
);

INSERT INTO COVID_VACCINATIONS_PROCESSED_V2
WITH total_vaccinations_portion AS (
    SELECT
        continent,
        location,
        date,
        total_vaccinations,
        SUM(CASE WHEN total_vaccinations IS NULL THEN 0 ELSE 1 END) OVER (PARTITION BY continent, location ORDER BY date) AS total_vaccinations_partition,
        new_vaccinations
    FROM COVID_VACCINATIONS_PROCESSED
),
new_total_vaccinations AS (
    SELECT
        continent,
        location,
        date,
        FIRST_VALUE(total_vaccinations) OVER(PARTITION BY total_vaccinations_partition, continent, location) AS total_vaccinations,
        new_vaccinations
    FROM total_vaccinations_portion
)

SELECT
    continent,
    location,
    date,
    total_vaccinations,
    total_vaccinations - LAG(total_vaccinations) OVER(PARTITION BY continent, location ORDER BY date) AS new_vaccinations
FROM new_total_vaccinations
ORDER BY location, date

 * sqlite:///covid_data.db
Done.
Done.
781572 rows affected.


[]

Let's once more review our processed table.

In [70]:
%%sql
SELECT *
FROM COVID_VACCINATIONS_PROCESSED_V2
LIMIT 10

 * sqlite:///covid_data.db
Done.


continent,location,date,total_vaccinations,new_vaccinations
Asia,Afghanistan,2020-01-05,None,None
Asia,Afghanistan,2020-01-05,None,None
Asia,Afghanistan,2020-01-06,None,None
Asia,Afghanistan,2020-01-06,None,None
Asia,Afghanistan,2020-01-07,None,None
Asia,Afghanistan,2020-01-07,None,None
Asia,Afghanistan,2020-01-08,None,None
Asia,Afghanistan,2020-01-08,None,None
Asia,Afghanistan,2020-01-09,None,None
Asia,Afghanistan,2020-01-09,None,None


Everything looks good. Let's delve into our analysis.

### Insights for Vietnam

In [71]:
# Let's look at the total population vs vaccinations
%%sql
SELECT
  dea.continent,
  dea.location,
  dea.date,
  dea.population,
  vac.total_vaccinations,
  vac.total_vaccinations / dea.population AS vaccin_percentage
FROM COVID_DEATHS_PROCESSED dea
JOIN COVID_VACCINATIONS_PROCESSED_V2 vac
  USING (location, date)
WHERE dea.continent IS NOT NULL
  AND dea.location == 'Vietnam'
  AND vac.new_vaccinations != 0
ORDER BY 1, 2, 3
LIMIT 10

 * sqlite:///covid_data.db
Done.


continent,location,date,population,total_vaccinations,vaccin_percentage
Asia,Vietnam,2021-03-08,98186856.0,377.0,3.839617799759267e-06
Asia,Vietnam,2021-03-08,98186856.0,377.0,3.839617799759267e-06
Asia,Vietnam,2021-03-09,98186856.0,522.0,5.316393876589754e-06
Asia,Vietnam,2021-03-09,98186856.0,522.0,5.316393876589754e-06
Asia,Vietnam,2021-03-10,98186856.0,955.0,9.726352781883555e-06
Asia,Vietnam,2021-03-10,98186856.0,955.0,9.726352781883555e-06
Asia,Vietnam,2021-03-11,98186856.0,1585.0,1.614269021914705e-05
Asia,Vietnam,2021-03-11,98186856.0,1585.0,1.614269021914705e-05
Asia,Vietnam,2021-03-12,98186856.0,5248.0,5.3449109318664814e-05
Asia,Vietnam,2021-03-12,98186856.0,5248.0,5.3449109318664814e-05


Here are some insights from the provided data on Vietnam's COVID-19 vaccination progress:

**General Trends**

**Initial Phase (March 2021 - May 2021):**
   - Vaccination efforts started very slowly in March 2021 with only a few hundred vaccinations per day.
   - By the end of May 2021, the total vaccinations had increased significantly but still remained below 1% of the population.

**Rapid Increase (June 2021 - December 2021):**
   - There was a sharp increase in vaccination rates from June 2021 onwards.
   - By the end of 2021, a substantial portion of the population had received vaccinations, crossing significant milestones regularly.

**Steady Growth (2022):**
   - The pace of vaccinations continued steadily throughout 2022.
   - By the end of 2022, a significant portion of the population had been vaccinated, with percentages reaching close to half of the population.

**Final Phase (January 2023 - June 2023):**
   - The rate of vaccinations continued to increase but at a slower rate compared to the initial rapid phase.
   - By June 30, 2023, a large majority of the population had received vaccinations, indicating a successful widespread vaccination campaign.

**Conclusion**

Vietnam's vaccination campaign showed a slow start but gained momentum in mid-2021, achieving significant coverage by the end of 2022 and continuing to expand in 2023. The data highlights the importance of sustained vaccination efforts to achieve widespread immunity within the population.

Lastly, we will create a view that combines the information from the `COVID_DEATHS` and `COVID_VACCINATIONS` tables for further visualization. Let's call it `PERCENT_POPULATION_VACCINATED_VIEW`

In [72]:
# Let's create a view to save data for later visualization
%%sql
DROP VIEW IF EXISTS PERCENT_POPULATION_VACCINATED_VIEW;

CREATE VIEW PERCENT_POPULATION_VACCINATED_VIEW
AS
SELECT
  dea.continent,
  dea.location,
  dea.date,
  dea.population,
  vac.total_vaccinations,
  vac.total_vaccinations / dea.population AS vaccin_percentage
FROM COVID_DEATHS_PROCESSED dea
JOIN COVID_VACCINATIONS_PROCESSED_V2 vac
  USING (location, date)
ORDER BY 1, 2, 3

 * sqlite:///covid_data.db
Done.
Done.


[]

In [73]:
# So now we can select from the view
%%sql
SELECT *
FROM PERCENT_POPULATION_VACCINATED_VIEW
LIMIT 10

 * sqlite:///covid_data.db
Done.


continent,location,date,population,total_vaccinations,vaccin_percentage
None,Africa,2020-01-05,1426736614.0,None,None
None,Africa,2020-01-05,1426736614.0,None,None
None,Africa,2020-01-05,1426736614.0,None,None
None,Africa,2020-01-05,1426736614.0,None,None
None,Africa,2020-01-06,1426736614.0,None,None
None,Africa,2020-01-06,1426736614.0,None,None
None,Africa,2020-01-06,1426736614.0,None,None
None,Africa,2020-01-06,1426736614.0,None,None
None,Africa,2020-01-07,1426736614.0,None,None
None,Africa,2020-01-07,1426736614.0,None,None


# Conclusion

This project delved into the COVID-19 dataset using advanced SQL techniques, revealing key insights into global trends, regional disparities, and vaccination progress. Through thorough analysis, it underscored the importance of data-driven decision-making in public health crises like COVID-19.

## Key Learnings

1. **SQL Proficiency**: Enhanced skills in advanced SQL techniques for handling complex datasets.
   
2. **Data Quality**: Addressed data quality issues for accurate analysis.

3. **Global Trends**: Identified significant global trends and disparities in COVID-19 cases, deaths, and vaccination rates.

4. **Regional Disparities**: Highlighted variations in the impact of COVID-19 across continents and income levels.

5. **Vaccination Progress**: Tracked the progress and challenges of global vaccination efforts.

6. **Visualization**: Emphasized the importance of data visualization for conveying insights effectively.

## Future Considerations

1. **Detailed Country Analysis**: Explore individual country factors influencing COVID-19 outcomes.

2. **Temporal Analysis**: Investigate the effectiveness of interventions over time.

3. **External Factors**: Integrate additional datasets to understand broader influences on COVID-19 trends.

4. **Predictive Modeling**: Use machine learning for forecasting future trends.

5. **Variant Impact**: Analyze the impact of COVID-19 variants on cases, deaths, and vaccine effectiveness.

6. **Long-term Effects**: Study the lasting health and economic impacts of COVID-19 for recovery and preparedness efforts.

This project lays the groundwork for ongoing analysis and decision-making in addressing the COVID-19 pandemic, emphasizing the value of data insights in public health responses.